In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

spark = SparkSession.builder.appName("SalesAnalysis_Lab").getOrCreate()

# Генерируем данные о покупках
data = [
    ("2023-01-01", "iPhone", "US", 1200.0, 1),
    ("2023-01-01", "Samsung", "KR", 900.0, 2),
    ("2023-01-01", "Xiaomi", "CN", 400.0, 5),
    ("2023-01-02", "iPhone", "US", 1200.0, 1),
    ("2023-01-02", "Samsung", "US", 900.0, 1), # Samsung продан в US
    ("2023-01-03", "Nokia", "FI", 50.0, 10),
    ("2023-01-03", "Xiaomi", "CN", 400.0, 0)  # Возврат (qty=0) или ошибка
]

schema = StructType([
    StructField("date", StringType(), True),
    StructField("product", StringType(), True),
    StructField("country", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("qty", IntegerType(), True)
])

df = spark.createDataFrame(data, schema=schema)
df.show()

+----------+-------+-------+------+---+
|      date|product|country| price|qty|
+----------+-------+-------+------+---+
|2023-01-01| iPhone|     US|1200.0|  1|
|2023-01-01|Samsung|     KR| 900.0|  2|
|2023-01-01| Xiaomi|     CN| 400.0|  5|
|2023-01-02| iPhone|     US|1200.0|  1|
|2023-01-02|Samsung|     US| 900.0|  1|
|2023-01-03|  Nokia|     FI|  50.0| 10|
|2023-01-03| Xiaomi|     CN| 400.0|  0|
+----------+-------+-------+------+---+



#### Задача 1: The Best Seller
1. Найдите самый популярный товар (по сумме проданных штук qty) для каждой страны.
2. Оставьте только те страны, где суммарная выручка превысила 2000.

Подсказка: Сначала groupBy("country", "product") для суммы штук, потом еще одна агрегация или фильтрация.

In [8]:
df_agg = df.groupBy(F.col("country"), F.col("product")).agg(
    F.sum(F.col("price")).alias("total_price"),
    F.sum(F.col("qty")).alias("total_cnt")
).orderBy(F.col("total_cnt").desc())
df_agg.show()

+-------+-------+-----------+---------+
|country|product|total_price|total_cnt|
+-------+-------+-----------+---------+
|     FI|  Nokia|       50.0|       10|
|     CN| Xiaomi|      800.0|        5|
|     US| iPhone|     2400.0|        2|
|     KR|Samsung|      900.0|        2|
|     US|Samsung|      900.0|        1|
+-------+-------+-----------+---------+



In [9]:
df_agg.filter(F.col("total_price") > 2000).show()

+-------+-------+-----------+---------+
|country|product|total_price|total_cnt|
+-------+-------+-----------+---------+
|     US| iPhone|     2400.0|        2|
+-------+-------+-----------+---------+



#### Задача 2: Уникальные пользователи (Approximate)
1. Представьте, что у вас 1 миллиард строк. countDistinct (точный подсчет уникальных) — очень дорогая операция (требует полной сортировки или огромного хэш-сета).
2. Используйте функцию F.approx_count_distinct("product").
3. Сравните результат с обычным countDistinct. В Big Data часто допускается погрешность 1-2% ради скорости x10.

In [16]:
df.select(F.count_distinct(F.col("product")).alias("unique_products")).show()
df.select(F.approx_count_distinct(F.col("product")).alias("aprox_unique_products")).show()


+---------------+
|unique_products|
+---------------+
|              4|
+---------------+

+---------------------+
|aprox_unique_products|
+---------------------+
|                    4|
+---------------------+

